In [1]:
# Nativos
import random as rn
import os
import sys
import gc
import time

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from utils_alertas import *

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]


In [2]:
data = pd.read_csv('AVANCE_6.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,_4_cut_up_DESFASE_CEF_MESES_REAL,_aux_TRX_CAIDA_SEGUIDA_PAGOS_RECIBIDOS__U3M,NRO_MESES_SIN_PAGO_DE_SERVICIOS__CASHOUT,TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MENOR_0.4,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5,_aux_TIENE_TRX_POS__U3M,F_Septile_cut_7.0,SOW_IBK_MIN_historia,MTO_ADEX_METAL_-_MECANICO_EXP,ADEX_CAIDA_40PERCENT_MES_ANT_IMP,...,_1_cut_up_NUM_DOCS_LAB_U12M,_3_cut_up_NUM_DOCS_TRIB_U1M,_0.5_cut_down_SOW_OTROSBCOS_MAX_historia,_3_cut_up_NUM_DOCS_PROT_U12M,_20percent_down_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,ACTIVIDAD_letal,_1_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,_1_cut_up_NUM_DOCS_IMPAG_U6M,_2_cut_up_NUM_DOCS_LAB_PROMEDIO_U9M,_2_cut_up_NUM_DOCS_PROT_U1M
0,0.0,0.0,3.0,0.0,0.0,0.0,0,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.105428,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0,0.083258,0.00,0.0,...,0.0,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0,0.000000,7305.01,0.0,...,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0
4,0.0,0.0,3.0,1.0,0.0,0.0,0,0.000000,0.00,0.0,...,0.0,0.0,0.0,1.0,1.0,0,0.0,1.0,0.0,1.0


In [3]:
'COD_UNICO' in data.columns, 'PERIODO' in data.columns

(True, True)

In [4]:
print(data.shape)
data = filtro_banca_s2provincia(data)
print(data.shape)

(24014, 3647)
(2082, 3647)


In [5]:
pd.crosstab(
    data['PERIODO'], data['ZONAL']
)

ZONAL,BEP ZONAL 1,BEP ZONAL 2,BEP ZONAL 3
PERIODO,,,
202009,144,124,91
202010,142,123,90
202011,142,117,93
202012,143,114,93
202101,140,106,87
202102,142,105,86


### CREACION DE COLUMNAS A MEZCLAR

In [6]:
cols_combine = [col for col in data.columns if set(data[col].unique()) == {1, 0} if '1_and_1' not in col]
print(len(cols_combine))

2367


In [7]:
cols_combine = \
    [col for col in cols_combine if 'SCOTIA' not in col.upper() \
                                 and 'BCP' not in col.upper() \
                                 and 'BBVA' not in col.upper() \
                                 and 'BIF' not in col.upper() \
                                 and '%' not in col.upper() \
                                 and 'COVID' not in col.upper() \
                                 and 'REFIN' not in col.upper() 
    ]
print(len(cols_combine))

2281


In [8]:
for col in ['cambio_seguir_covid', 'TIENE_DIAS_DE_ATRASO_IBK_ESTE_MES'] + [col for col in data.columns if 'target' in col.lower()]:
    if col in cols_combine:
        cols_combine.remove(col)

len(cols_combine)

2280

### GENERACION DE ALERTAS

In [9]:
existentes = pd.read_csv('COMBINACIONES_S2_PROVINCIA.csv', sep=',').sort_values(by=['SCORE'], ascending=False)
existentes.head(50)

,VAR_1,VAR_2,VAR_3,CASOS,CORRECTOS,SCORE
6919,_85_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION,_1_cut_up_NUM_DOCS_LAB_U1M,NaN,6,6,1.000000
6056,_110_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION,_1_cut_up_NUM_DOCS_LAB_U1M,NaN,6,6,1.000000
6190,_1_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,_55_cut_up_ROTACION_CTASXCOBRAR_VAR_DIAS_CEF_S...,NaN,6,6,1.000000
6609,TIENE_AUMENTO_COL_DIRECTA_10PERCENT_A_MAS_U7M,_120_cut_up_ROTACION_CTASXPAGAR_VAR_DIAS_CEF_S...,NaN,5,5,1.000000
5189,_55_cut_up_ROTACION_CTASXCOBRAR_VAR_DIAS_CEF_S...,_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYO...,NaN,6,6,1.000000
5503,_1_cut_up_NUM_DOCS_LAB_U1M,_110_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION,NaN,6,6,1.000000
7784,IMPORTE_LAB_REGISTRADO_MAYOR_A_2.5MIL_SOLES,_110_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION,NaN,5,5,1.000000
6650,_110_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION,IMPORTE_LAB_REGISTRADO_MAYOR_A_2.5MIL_SOLES,NaN,5,5,1.000000
7824,TIENE_MENOS_1PERCENT_GARTOTAL_IBK,_25000_cut_up_MTO_ATRASO_NOLETRAS,NaN,8,7,0.875000
6065,DEBT_NO_MTOSALDO_IBK_SF_U3M,_2_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,NaN,8,7,0.875000


In [10]:
#existentes.to_csv('COMBINACIONES_S2_PROVINCIA.csv', index=False)

In [11]:
try:
    existentes['union'] = existentes[['VAR_1', 'VAR_2', 'VAR_3']].apply(
        lambda _: '1_and_1_{}___{}'.format(_[0], _[1]) if pd.isnull(_[2]) else '1_and_1_and_1_{}___{}____{}'.format(_[0], _[1], _[2]), 
        axis=1
    )
    listado = list(existentes['union'].unique())
    len(listado)
except:
    listado = []
    
len(listado)

8026

In [12]:
import random as rn
import itertools

col_target = 'target_FINAL_fusion_atraso_feve_2Mbueno'
display(pd.crosstab(
    data[col_target].fillna(-1), data['PERIODO']
))

aucc = {}
struct = []

data = data[data[col_target].notnull()][cols_combine + ['COD_UNICO', col_target, 'PERIODO']]
data.shape

PERIODO,202009,202010,202011,202012,202101,202102
target_FINAL_fusion_atraso_feve_2Mbueno,,,,,,
-1.0,136,128,129,120,98,88
0.0,192,206,207,213,218,227
1.0,31,21,16,17,17,18


(1383, 2283)

In [13]:
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_MEDIANA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S1_PROVINCIA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S2_PROVINCIA.csv', index=False)

In [14]:
#['TIENE_MENOS_5PERCENT_GARTOTAL_IBK', 'MORE_0.11_PERCENT_DELAY_OVER_TOTAL_DEBT_CC', None, 12, 9, 0.75]

In [15]:
SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

rn.shuffle(cols_combine)

In [16]:
over_interes = [col for col in data.columns if ('OVER_TRIM_PASADO_MAYOR' in col.upper() or 'CIERRE' in col.upper()) and '1_and_1' not in col]
over_interes

['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.2',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.1',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.4',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.5',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.8',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.6',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.7',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.3',
 'TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.9',
 '_2500000_cut_up_RIESGO_DIRECTO_CIERRE_MES',
 '_2700000_cut_up_RIESGO_DIRECTO_CIERRE_MES',
 '_1000000_cut_up_RIESGO_DIRECTO_CIERRE_MES',
 '_2000000_cut_up_RIESGO_DIRECTO_CIERRE_MES',
 '_1500000_cut_up_RIESGO_DIRECTO_CIERRE_MES']

In [ ]:
from datetime import datetime
tiempo_inicio = time.time()
nn = len(struct)

while True:
    x = rn.choice(over_interes)
    y = rn.choice(cols_combine)
    z = rn.choice(cols_combine)
    
    col_evaluate = '1_and_1_and_1_{}___{}____{}'.format(x, y, z)
    
    if col_evaluate not in listado and x != y and x!= z and y!= z:
        listado.append(col_evaluate)
        
        lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1 and _[2] >= 1
        data[col_evaluate] = data[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        #data_play_1[col_evaluate] = data_play_1[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        
        
        coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
        acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
        
        if acertados > 6 and acertados/coincidencias > 0.35:
            aucc[col_evaluate] = acertados/coincidencias
            struct.append(
                [x, y, z, coincidencias, acertados, acertados/coincidencias]
            )
            tiempo_inicio = time.time()
            nn = len(struct)
            #print(struct[-1])
            
        ### Generado pares de 2 
        for a, b in list(itertools.combinations([x, y, z], 2)):
            
            col_evaluate = '1_and_1_{}___{}'.format(a, b)
            if col_evaluate not in listado:
                lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1
                data[col_evaluate] = data[[a, b]].apply(lambda_basico, axis=1).astype(int)
                #data_play_1[col_evaluate] = data_play_1[[a, b]].apply(lambda_basico, axis=1).astype(int)
                
                coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
                acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
                
                if acertados > 6 and acertados/coincidencias > 0.35:
                    aucc[col_evaluate] = acertados/coincidencias
                    struct.append(
                        [a, b, None, coincidencias, acertados, acertados/coincidencias]
                    )
                    tiempo_inicio = time.time()
                    nn = len(struct)
                    #print(struct[-1])
        
        if len(struct) >= 5:
            pd.DataFrame(struct).to_csv('COMBINACIONES_S2_PROVINCIA.csv', header=False, index=False, mode='a')
            print("/"*25, datetime.now())
            struct = []
            
        if time.time() - tiempo_inicio >= 3600 and nn == len(struct):
            print("Mas de 1 hora sin nueva combinacion de alerta")
            break


///////////////////////// 2021-05-29 14:55:55.726682
///////////////////////// 2021-05-29 14:57:30.983378
///////////////////////// 2021-05-29 14:59:57.936651
///////////////////////// 2021-05-29 15:02:04.143769
///////////////////////// 2021-05-29 15:03:55.896566
///////////////////////// 2021-05-29 15:08:17.028064
///////////////////////// 2021-05-29 15:10:26.055609
///////////////////////// 2021-05-29 15:15:22.073546
///////////////////////// 2021-05-29 15:16:59.299889
///////////////////////// 2021-05-29 15:18:22.788799
///////////////////////// 2021-05-29 15:24:37.621963
///////////////////////// 2021-05-29 15:26:22.729837
///////////////////////// 2021-05-29 15:30:54.449611
///////////////////////// 2021-05-29 15:34:27.631271
///////////////////////// 2021-05-29 15:38:54.143163
///////////////////////// 2021-05-29 15:42:54.668897
///////////////////////// 2021-05-29 15:46:49.030811
///////////////////////// 2021-05-29 15:53:04.615115
///////////////////////// 2021-05-29 15:59:04.